# Faces 

In [7]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

## Faces decection with haar cascades

Detect the presence of the face in an image


Haar Cascade is really sensitive to noise


Increasing scaleFactor and minNeighbors will reduce this


Decreasing them will make easier to find 

In [6]:
image = cv.imread('Resources/Photos/group 2.jpg')
gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

# haar cascade classifier
haar_cascade = cv.CascadeClassifier('haar_face.xml')

faces_rect = haar_cascade.detectMultiScale(gray,scaleFactor = 1.1, minNeighbors = 6)

print(f'Number of faces found = {len(faces_rect)}') # number of faces find in the image

for (x,y,w,h) in faces_rect:
    cv.rectangle(image,(x,y),(x+w,y+h),color = (0,255,0),thickness = 2)

# display
cv.imshow('Groupe',image)  

# wait time for a key to be press
cv.waitKey(0)

Number of faces found = 7


-1

## Face Recognition with openCV built-in recognizer

Identify whose face it is

### Training and saving model

In [15]:
# create list with all people name
people = []
dir = '/home/alessandrini/Documents/Formation/Python/Computer_Vision_Python/Resources/Faces/train'
for name in os.listdir(dir):
    people.append(name)

# haar cascade classifier
haar_cascade = cv.CascadeClassifier('haar_face.xml')

features = []
labels = []

# add every image of our training set
def create_train(folder):
    for person in people:
        path = os.path.join(folder,person)
        label = people.index(person)
        
        for image in os.listdir(path):
            image_path = os.path.join(path,image)

            #load image
            image_array = cv.imread(image_path)
            gray = cv.cvtColor(image_array,cv.COLOR_BGR2GRAY)

            # detect face
            faces_rect = haar_cascade.detectMultiScale(gray,scaleFactor = 1.1, minNeighbors = 4)

            for (x,y,w,h) in faces_rect:
                faces_roi = gray[y:y+h,x:x+w]
                features.append(faces_roi)
                labels.append(label)

create_train(dir)
print('Loading training done')
print(f'length of the features = {len(features)}')
print(f'length of the labels = {len(labels)}')

# initiate face recognize
face_recognizer = cv.face.LBPHFaceRecognizer_create()

# Train the recognizer on the feature and the labels (transform in np array)
features = np.array(features,dtype='object')
labels = np.array(labels)
face_recognizer.train(features,labels)

# save features, labels and recognizer model
np.save('features.npy',features)
np.save('labels.npy',labels)
face_recognizer.save('face_trained.yml')



Loading training done
length of the features = 100
length of the labels = 100


### Loading and using model in an other file

In [19]:
# haar cascade classifier
haar_cascade = cv.CascadeClassifier('haar_face.xml')

# load recognizer
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read('face_trained.yml')

# create list with all people name
people = []
dir = '/home/alessandrini/Documents/Formation/Python/Computer_Vision_Python/Resources/Faces/train'
for name in os.listdir(dir):
    people.append(name)

image = cv.imread('Resources/Faces/val/ben_afflek/2.jpg')
gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

# detect the face
faces_rect = haar_cascade.detectMultiScale(gray,scaleFactor = 1.1, minNeighbors = 4)
for (x,y,w,h) in faces_rect:
    faces_roi = gray[y:y+h,x:x+w]
    label, confidence = face_recognizer.predict(faces_roi)

    print(f'Label = {people[label]} with confidence of {confidence}')
    
    cv.putText(image,str(people[label]),(20,20),cv.FONT_HERSHEY_COMPLEX,1.0,(0,255,0),2)
    cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)

cv.imshow('Detected Face', image)
cv.waitKey(0)

Label = Ben Afflek with confidence of 60.069416291380506


-1